In [21]:
import pytz
import datetime
import pandas as pd
import collections
import re
from pymongo import MongoClient
import numpy as np

In [22]:
def removeSpecialCharacters(value):
    return re.sub('[^A-Za-z0-9]+', ' ', str(value))

def cleanDataColumns(data):
    trueCol = []
    for col in data.columns:
        trueCol.append(removeSpecialCharacters(col))
    data.columns = trueCol
    return data

def limpa_cpf(cpfString):
    cpfString = cpfString.replace('.','')
    cpfString = cpfString.replace('-','')
    return int(cpfString)    

def limpeza_de_cpfs(dados, coluna):
    dados[coluna] = dados.loc[:,coluna].apply(limpa_cpf)
    return dados
            
        
def coluna_para_datetime(data, coluna):
    data[coluna] = pd.to_datetime(data[coluna], format='%d/%m/%Y')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data

def string_to_datetime(datetime_string):
    datetime_string = datetime_string.split('/')
    datetime_value = datetime.datetime(
        int(datetime_string[2]),
        int(datetime_string[1]),
        int(datetime_string[0]))
    return date_to_Brazil_East_Utc(datetime_value)

def date_to_Brazil_East_Utc(datetime_value): # ele adianta o relogio em 3 horas
    if str(datetime_value) == 'NaT':
        datetime_value = datetime.datetime(1500,1,1) # minha data nulla e 01/01/1500
    local = pytz.timezone("Brazil/East")
    local_dt = local.localize(datetime_value, is_dst=True) # esse is_dst e pra ambiguidades, deve ser horario de verao aqui
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt.strftime ("%Y-%m-%d %H:%M:%S")

def float_latin_nan_e_zero(entry):
    if _isNan(entry):
        return 0.0e0
    else:
        return floatLatin(entry)
    
def floatComVirgula(entry):
    if _isNan(entry):
        return np.nan
    else:
        entry = str(entry).replace(',','')
        return float(entry)
        
def floatLatin(entry):
    if _isNan(entry):
        return np.nan
    entry = str(entry).replace('.','')
    entry = str(entry).replace(',','.')
    return float(entry)

def _isNan(value):
    if isinstance(value, str):
        return value == 'nan'
    else:
        return np.isnan(value)


# TRANSFORMACOES ESPECIFICAS DA TABELA ACORDOS

def transforma_Assessoria(valor):
    if str(valor) == 'nan':
        return 'SEMEAR'
    else:
        return str(valor)

def transforma_Cobrador(valor):
    if 'Semear_Renego' in valor:
        return 'Renegociacao'
    elif 'Multiparcelas' in valor:
        return 'Multiparcelas'
    else:
        return 'NULO'

# TRANSFORMACOES ESPECIFICAS DA TABELA ACIONAMENTOS

def coluna_para_datetime_hora(data, coluna):
    data[coluna] = pd.to_datetime(data[coluna], format='%d/%m/%Y %H:%M')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data

# TRANSFORMACOES ESPECIFICAS DO RELATORIO 3040

def coluna_para_datetime_hora_seg(data, coluna):
    data[coluna] = pd.to_datetime(data[coluna], format='%Y-%m-%d %H:%M:%S')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data

def string_para_datetime(datetime_string):
    return datetime.datetime.strptime(datetime_string, '%Y-%m-%d %H:%M:%S')


In [23]:
dados_01 = pd.read_csv('3040_01_2019.csv', sep=';', encoding='latin-1', dtype=str)
dados_02 = pd.read_csv('3040_02_2019.csv', sep=';', encoding='latin-1', dtype=str)
dados_03 = pd.read_csv('3040_03_2019.csv', sep=';', encoding='latin-1', dtype=str)
dados_04 = pd.read_csv('3040_04_2019.csv', sep=';', encoding='latin-1', dtype=str)
dados_05 = pd.read_csv('3040_05_2019.csv', sep=';', encoding='latin-1', dtype=str)
dados_06 = pd.read_csv('3040_06_2019.csv', sep=';', encoding='latin-1', dtype=str)

In [24]:
acionamentos = pd.read_csv('Acionamentos_01012019_a_30062019.csv', sep=';', encoding='latin-1', dtype=str)
acionamentos = cleanDataColumns(acionamentos)
cpf_null = acionamentos[acionamentos.loc[:,'CPF CNPJ'].isnull()].index
acionamentos = acionamentos.drop(cpf_null)
acionamentos = acionamentos.reset_index(drop=True)
datahora = []
for i in acionamentos.index:
    datahora.append(acionamentos.loc[i,'Data'] + ' ' + acionamentos.loc[i,'Hor rio'])
    
acionamentos['datahora'] = datahora
acionamentos = coluna_para_datetime_hora(acionamentos, 'datahora')
acionamentos['datahora'] = [string_para_datetime(x) for x in acionamentos.loc[:,'datahora'].tolist()]

In [25]:
dados_01 = cleanDataColumns(dados_01)
dados_02 = cleanDataColumns(dados_02)
dados_03 = cleanDataColumns(dados_03)
dados_04 = cleanDataColumns(dados_04)
dados_05 = cleanDataColumns(dados_05)
dados_06 = cleanDataColumns(dados_06)

In [ ]:
data_inicio = datetime.datetime.strptime('2019-02-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2019-02-28 23:59:59', '%Y-%m-%d %H:%M:%S')
intervalo_de_acionamentos = acionamentos[
        (acionamentos.loc[:,'datahora'] < data_fim) &
        (acionamentos.loc[:,'datahora'] > data_inicio)].copy()
print('Numero de acionamentos neste intervalo:  ', len(set(intervalo_de_acionamentos.loc[:,'CPF CNPJ'])))

In [ ]:
data_inicio = datetime.datetime.strptime('2019-03-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2019-03-31 23:59:59', '%Y-%m-%d %H:%M:%S')
intervalo_de_acionamentos = acionamentos[
        (acionamentos.loc[:,'datahora'] < data_fim) &
        (acionamentos.loc[:,'datahora'] > data_inicio)].copy()
print('Numero de acionamentos neste intervalo:  ', len(set(intervalo_de_acionamentos.loc[:,'CPF CNPJ'])))

In [ ]:
data_inicio = datetime.datetime.strptime('2019-04-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2019-04-30 23:59:59', '%Y-%m-%d %H:%M:%S')
intervalo_de_acionamentos = acionamentos[
        (acionamentos.loc[:,'datahora'] < data_fim) &
        (acionamentos.loc[:,'datahora'] > data_inicio)].copy()
print('Numero de acionamentos neste intervalo:  ', len(set(intervalo_de_acionamentos.loc[:,'CPF CNPJ'])))

In [ ]:
data_inicio = datetime.datetime.strptime('2019-05-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2019-05-31 23:59:59', '%Y-%m-%d %H:%M:%S')
intervalo_de_acionamentos = acionamentos[
        (acionamentos.loc[:,'datahora'] < data_fim) &
        (acionamentos.loc[:,'datahora'] > data_inicio)].copy()
print('Numero de acionamentos neste intervalo:  ', len(set(intervalo_de_acionamentos.loc[:,'CPF CNPJ'])))

In [ ]:
data_inicio = datetime.datetime.strptime('2019-06-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2019-06-30 23:59:59', '%Y-%m-%d %H:%M:%S')
intervalo_de_acionamentos = acionamentos[
        (acionamentos.loc[:,'datahora'] < data_fim) &
        (acionamentos.loc[:,'datahora'] > data_inicio)].copy()
print('Numero de acionamentos neste intervalo:  ', len(set(intervalo_de_acionamentos.loc[:,'CPF CNPJ'])))

In [ ]:
data_inicio = datetime.datetime.strptime('2019-07-01 00:00:00', '%Y-%m-%d %H:%M:%S')
data_fim = datetime.datetime.strptime('2019-07-30 23:59:59', '%Y-%m-%d %H:%M:%S')
intervalo_de_acionamentos = acionamentos[
        (acionamentos.loc[:,'datahora'] < data_fim) &
        (acionamentos.loc[:,'datahora'] > data_inicio)].copy()
print('Numero de acionamentos neste intervalo:  ', len(set(intervalo_de_acionamentos.loc[:,'CPF CNPJ'])))

In [26]:
def corte_pdd(dados):
    dados.VlrPDD = dados.VlrPDD.apply(floatComVirgula)
    dados_corte = dados[dados.VlrPDD > 1e-5].copy() # Remove aquels VlrPDD 0
    dados_corte.DiasAtraso = dados_corte.DiasAtraso.astype(int)
    numero_5 = len(set(dados_corte[dados_corte.DiasAtraso >= 5].CpfCgc))
    numero_30 = len(set(dados_corte[dados_corte.DiasAtraso >= 30].CpfCgc))
    numero_60 = len(set(dados_corte[dados_corte.DiasAtraso >= 60].CpfCgc))
    print('5 dias: ', numero_5,
         '  30 dias: ', numero_30,
         '  60 dias: ', numero_60)
    

In [27]:
corte_pdd(dados_06) # CORTES OBTIDOS AQUI

5 dias:  62555   30 dias:  30294   60 dias:  19414


In [28]:
dados_05.VlrPDD = dados_05.VlrPDD.apply(floatComVirgula)
dados_corte = dados_05[dados_05.VlrPDD > 1e-5].copy()

In [ ]:
dados_corte.DiasAtraso = dados_corte.DiasAtraso.astype(int)
dados_corte[dados_corte.DiasAtraso >= 5]

In [ ]:
len(set(dados_corte[dados_corte.DiasAtraso >= 5].CpfCgc))

In [ ]:
len(set(dados_corte[dados_corte.DiasAtraso >= 30].CpfCgc))

In [ ]:
len(set(dados_corte[dados_corte.DiasAtraso >= 60].CpfCgc))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


labels = ['Fev', 'Mar', 'Abr', 'Mai', 'Jun']
acionam = [23, 6129, 13687, 17129, 25020]
atraso_5 = [63061, 66732, 67356, 60978, 62555]
atraso_30 = [30909, 33727, 34968, 31584, 30294]
atraso_60 = [18932, 20258, 23307, 19646, 19414]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars
r1 = x
r2 = [x + width for x in r1]
r3 = [x + width for x in r2]


fig, ax = plt.subplots()
rects1 = ax.bar(r1, acionam, width, label='Acionamentos')
rects2 = ax.bar(r2, atraso_60, width, label='Atraso 60')
rects3 = ax.bar(r3, atraso_30, width, label='Atraso 30')
#rects4 = ax.bar(xPlot + width/2, atraso_5, width, label='Atraso 5')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Número')
ax.set_title('CPFs acionados por CPFs em atraso')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


#autolabel(rects1)
#autolabel(rects2)
#autolabel(rects3)
#autolabel(rects4)

fig.tight_layout()

plt.show()

In [ ]:
# libraries
import numpy as np
import matplotlib.pyplot as plt

figura = plt.figure()

# set width of bar
barWidth = 0.2
 
# set height of bar
labels = ['Fev', 'Mar', 'Abr', 'Mai', 'Jun']
acionam = [23, 6129, 13687, 17129, 25020]
atraso_5 = [63061, 66732, 67356, 60978, 62555]
atraso_30 = [30909, 33727, 34968, 31584, 30294]
atraso_60 = [18932, 20258, 23307, 19646, 19414]

bars1 = acionam
bars2 = atraso_60
bars3 = atraso_30
bars4 = atraso_5
 
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
 
# Make the plot
plt.bar(r1, bars1, color='k', width=barWidth, edgecolor='white', label='Acionamentos')
plt.bar(r2, bars2, color='b', width=barWidth, edgecolor='white', label='Atraso 60 dias')
plt.bar(r3, bars3, color='g', width=barWidth, edgecolor='white', label='Atraso 30 dias')
plt.bar(r4, bars4, color='r', width=barWidth, edgecolor='white', label='Atraso 5 dias')
 
# Add xticks on the middle of the group bars
plt.xlabel('ACIONAMENTOS VS CPFs ATRASADOS')
plt.ylabel('Numero de CPFs')
plt.xticks([r + barWidth for r in range(len(bars1))], labels)
 
# Create legend & Show graphic
plt.legend()
figura.savefig('sada',dpi=150)
#plt.show()

# RELATORIO DE ROLAGEM

In [ ]:
dados_01.DiasAtraso = dados_01.DiasAtraso.astype(int)
dados_02.DiasAtraso = dados_02.DiasAtraso.astype(int)
dados_03.DiasAtraso = dados_03.DiasAtraso.astype(int)
dados_04.DiasAtraso = dados_04.DiasAtraso.astype(int)
dados_05.DiasAtraso = dados_05.DiasAtraso.astype(int)
dados_06.DiasAtraso = dados_06.DiasAtraso.astype(int)
dados_01.CodProduto = dados_01.CodProduto.astype(int)
dados_02.CodProduto = dados_02.CodProduto.astype(int)
dados_03.CodProduto = dados_03.CodProduto.astype(int)
dados_04.CodProduto = dados_04.CodProduto.astype(int)
dados_05.CodProduto = dados_05.CodProduto.astype(int)
dados_06.CodProduto = dados_06.CodProduto.astype(int)

In [ ]:
CODIGOS_EP = [3064, 3065, 3069, 3070, 3071, 3072, 3077, 3078, 3081, 3086, 3087, 3088, 3092, 3095, 3096, 3103, 3105]

def Numeros_de_rolagem(dados):    
    rol_1_30 = len(dados[(dados.DiasAtraso >= 1) & (dados.DiasAtraso <= 30)])
    rol_31_60 = len(dados[(dados.DiasAtraso >= 31) & (dados.DiasAtraso <= 60)])
    rol_61_90 = len(dados[(dados.DiasAtraso >= 61) & (dados.DiasAtraso <= 90)])
    rol_91_120 = len(dados[(dados.DiasAtraso >= 91) & (dados.DiasAtraso <= 120)])
    rol_121_150 = len(dados[(dados.DiasAtraso >= 121) & (dados.DiasAtraso <= 150)])
    rol_151_180 = len(dados[(dados.DiasAtraso >= 151) & (dados.DiasAtraso <= 180)])
    return [rol_1_30, rol_31_60, rol_61_90, rol_91_120, rol_121_150, rol_151_180]

def Numeros_de_rolagem_EP(dados):    
    dados_EP = dados[dados.CodProduto.isin(CODIGOS_EP)].copy()
    rol_1 = len(dados_EP[(dados_EP.DiasAtraso >= 0) & (dados_EP.DiasAtraso <= 5)])
    rol_2 = len(dados_EP[(dados_EP.DiasAtraso >= 6) & (dados_EP.DiasAtraso <= 30)])
    rol_3 = len(dados_EP[(dados_EP.DiasAtraso >= 31) & (dados_EP.DiasAtraso <= 60)])
    rol_4 = len(dados_EP[(dados_EP.DiasAtraso >= 61) & (dados_EP.DiasAtraso <= 90)])
    rol_5 = len(dados_EP[(dados_EP.DiasAtraso >= 91) & (dados_EP.DiasAtraso <= 120)])
    rol_6 = len(dados_EP[(dados_EP.DiasAtraso >= 121) & (dados_EP.DiasAtraso <= 150)])
    rol_7 = len(dados_EP[(dados_EP.DiasAtraso >= 151) & (dados_EP.DiasAtraso <= 180)])
    rol_8 = len(dados_EP[dados_EP.DiasAtraso >= 181])
    return [rol_1, rol_2, rol_3, rol_4, rol_5, rol_6, rol_7, rol_8]


In [ ]:
rolagens = []
rolagens.append(Numeros_de_rolagem(dados_01))
rolagens.append(Numeros_de_rolagem(dados_02))
rolagens.append(Numeros_de_rolagem(dados_03))
rolagens.append(Numeros_de_rolagem(dados_04))
rolagens.append(Numeros_de_rolagem(dados_05))
rolagens.append(Numeros_de_rolagem(dados_06))
pd.DataFrame(data=rolagens).to_csv('relatorio_de_rolagens.csv',sep=';')

In [ ]:
rolagens = []
rolagens.append(Numeros_de_rolagem_EP(dados_01))
rolagens.append(Numeros_de_rolagem_EP(dados_02))
rolagens.append(Numeros_de_rolagem_EP(dados_03))
rolagens.append(Numeros_de_rolagem_EP(dados_04))
rolagens.append(Numeros_de_rolagem_EP(dados_05))
rolagens.append(Numeros_de_rolagem_EP(dados_06))
pd.DataFrame(data=rolagens).to_csv('relatorio_de_rolagens_EP.csv',sep=';')

# RELATORIO DE ROLAGEM FINANCEIRO

In [ ]:
# Definicao do valor em aberto:
# VlrProxParcela * (1 + diasEmAtraso / 30)
# Se, por exemplo, o valor da parcela for 200 reais, temos:
# Se, DiasEmAtraso = 10, entao, valor em aberto = 200.
# Se, DiasEmAtraso = 40, entao, valor em aberto = 400.
# tem que olhar a quantidade de parcelas abertas tambem

CODIGOS_EP = [3064, 3065, 3069, 3070, 3071, 3072, 3077, 3078, 3081, 3086, 3087, 3088, 3092, 3095, 3096, 3103, 3105]

def calcule_perda_financeira(dados, meses_em_atraso_base):
    perda = []
    cheio = str(meses_em_atraso_base) == 'CHEIO'
    for i in dados.index:
        meses_em_atraso_I = dados.QtdParAberta[i]
        if cheio: 
            pass
        elif meses_em_atraso_I >= meses_em_atraso_base and not cheio:
            meses_em_atraso_I = meses_em_atraso_base
        perda_financeira = meses_em_atraso_I * dados.VlrProxParcela[i]
        perda.append(perda_financeira)
    return sum(perda)
    
def Numeros_de_rolagem_EP_financeiro(dados):    
    #rol_1 = dados[(dados.DiasAtraso >= 0) & (dados.DiasAtraso <= 5)]
    #rol_1 = 0
    rol_2 = dados[(dados.DiasAtraso >= 6) & (dados.DiasAtraso <= 30)]
    rol_2 = calcule_perda_financeira(rol_2, 1)
    rol_3 = dados[(dados.DiasAtraso >= 31) & (dados.DiasAtraso <= 60)]
    rol_3 = calcule_perda_financeira(rol_3, 2)
    rol_4 = dados[(dados.DiasAtraso >= 61) & (dados.DiasAtraso <= 90)]
    rol_4 = calcule_perda_financeira(rol_4, 3)
    rol_5 = dados[(dados.DiasAtraso >= 91) & (dados.DiasAtraso <= 120)]
    rol_5 = calcule_perda_financeira(rol_5, 4)
    rol_6 = dados[(dados.DiasAtraso >= 121) & (dados.DiasAtraso <= 150)]
    rol_6 = calcule_perda_financeira(rol_6, 5)
    rol_7 = dados[(dados.DiasAtraso >= 151) & (dados.DiasAtraso <= 180)]
    rol_7 = calcule_perda_financeira(rol_7, 6)
    rol_8 = dados[dados.DiasAtraso >= 181]
    rol_8 = calcule_perda_financeira(rol_8, 'CHEIO')
    return [0, rol_2, rol_3, rol_4, rol_5, rol_6, rol_7, rol_8]

def rolagem_financeira(dados):
    dados = cleanDataColumns(dados)
    dados.CodProduto = dados.CodProduto.astype(int)
    dados_EP = dados[dados.CodProduto.isin(CODIGOS_EP)].copy()
    dados_EP.DiasAtraso = dados_EP.DiasAtraso.astype(int)
    dados_EP.VlrProxParcela = dados_EP.VlrProxParcela.apply(floatComVirgula)
    dados_EP.QtdParAberta = dados_EP.QtdParAberta.astype(int)
    return Numeros_de_rolagem_EP_financeiro(dados_EP)


In [ ]:
rolagem_todos = []
dados = pd.read_excel('3040_01_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_02_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_03_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_04_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_05_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_06_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))

In [ ]:
pd.DataFrame(rolagem_todos).to_csv('rolagem_financeira.csv',sep=';')

# FINANCEIRO COM PERDA CHEIA

In [18]:
# Definicao do valor em aberto:
# VlrProxParcela * (1 + diasEmAtraso / 30)
# Se, por exemplo, o valor da parcela for 200 reais, temos:
# Se, DiasEmAtraso = 10, entao, valor em aberto = 200.
# Se, DiasEmAtraso = 40, entao, valor em aberto = 400.
# tem que olhar a quantidade de parcelas abertas tambem

CODIGOS_EP = [3064, 3065, 3069, 3070, 
              3071, 3072, 3077, 3078, 
              3081, 3086, 3087, 3088, 
              3092, 3095, 3096, 3103, 3105]

def calcule_perda_financeira(dados, meses_em_atraso_base):
    perda = []
    cheio = str(meses_em_atraso_base) == 'CHEIO'
    for i in dados.index:
        meses_em_atraso_I = dados.QtdParAberta[i]
        if cheio: 
            pass
        elif meses_em_atraso_I >= meses_em_atraso_base and not cheio:
            meses_em_atraso_I = meses_em_atraso_base
        perda_financeira = meses_em_atraso_I * dados.VlrProxParcela[i]
        perda.append(perda_financeira)
    return sum(perda)
    
def Numeros_de_rolagem_EP_financeiro(dados):    
    rol_1 = dados[(dados.DiasAtraso >= 0) & (dados.DiasAtraso <= 5)]
    rol_1 = calcule_perda_financeira(rol_1, 'CHEIO')
    rol_2 = dados[(dados.DiasAtraso >= 6) & (dados.DiasAtraso <= 30)]
    rol_2 = calcule_perda_financeira(rol_2, 'CHEIO')
    rol_3 = dados[(dados.DiasAtraso >= 31) & (dados.DiasAtraso <= 60)]
    rol_3 = calcule_perda_financeira(rol_3, 'CHEIO')
    rol_4 = dados[(dados.DiasAtraso >= 61) & (dados.DiasAtraso <= 90)]
    rol_4 = calcule_perda_financeira(rol_4, 'CHEIO')
    rol_5 = dados[(dados.DiasAtraso >= 91) & (dados.DiasAtraso <= 120)]
    rol_5 = calcule_perda_financeira(rol_5, 'CHEIO')
    rol_6 = dados[(dados.DiasAtraso >= 121) & (dados.DiasAtraso <= 150)]
    rol_6 = calcule_perda_financeira(rol_6, 'CHEIO')
    rol_7 = dados[(dados.DiasAtraso >= 151) & (dados.DiasAtraso <= 180)]
    rol_7 = calcule_perda_financeira(rol_7, 'CHEIO')
    rol_8 = dados[dados.DiasAtraso >= 181]
    rol_8 = calcule_perda_financeira(rol_8, 'CHEIO')
    return [rol_1, rol_2, rol_3, rol_4, rol_5, rol_6, rol_7, rol_8]

def rolagem_financeira(dados):
    dados = cleanDataColumns(dados)
    dados.CodProduto = dados.CodProduto.astype(int)
    dados_EP = dados[dados.CodProduto.isin(CODIGOS_EP)].copy()
    dados_EP.DiasAtraso = dados_EP.DiasAtraso.astype(int)
    dados_EP.VlrProxParcela = dados_EP.VlrProxParcela.apply(floatComVirgula)
    dados_EP.QtdParAberta = dados_EP.QtdParAberta.astype(int)
    return Numeros_de_rolagem_EP_financeiro(dados_EP)


In [19]:
rolagem_todos = []
dados = pd.read_excel('3040_01_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_02_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_03_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_04_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_05_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))
dados = pd.read_excel('3040_06_2019.xlsx', sep=';', encoding='latin-1', dtype=str)
rolagem_todos.append(rolagem_financeira(dados))

In [20]:
pd.DataFrame(rolagem_todos).to_csv('rolagem_financeira_2.csv',sep=';')